# <div align="center">Training Omedena Abuja ANPD</div>

<p align="center">
Training of Omdena Abuja ANPD should be done in a GPU enabled environment.
</p>

<div div align="center">
   <a href="https://colab.research.google.com/github/abdulazeezoj/omdena-abuja-anpd/blob/master/notebook/Train_ANPD.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</div>

----

## 1. Cloning and Building Darknet

The Omdena Abuja ANPD is trained on YOLOv4-Tiny. You only look once (YOLO) is a state-of-the-art, real-time object detection system. In the following cells *darknet* from *AlexeyAB's* famous repository will be cloned, you will adjust the Makefile to enable OPENCV and GPU for *darknet* and then build *darknet*.

In [ ]:
# clone darknet repository

!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# verify CUDA

!nvcc --version

In [ ]:
# change makefile to have GPU and OPENCV enabled

%cd darknet/

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# builds darknet

!make

# cd out of darknet dir
%cd ../  

## 2. Download Omdena Abuja ANPD Config

The Omdena Abuja ANPD Config contains configuration files required to train the ANPD and YOLOv4-Tiny weights file that has been trained already on the coco dataset which has 80 classes that it can predict. We will grab the config so that we can run YOLOv4 on these pretrained weights and get detections.

In [ ]:
# download and unzip config file

!wget https://github.com/abdulazeezoj/omdena-abuja-anpd/releases/download/v0.1.0-alpha/anpd.zip
!unzip anpd.zip

%cd anpd
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

%cd ../

## 3. Define Helper Functions

The following functions are helper functions that will allow you to show the image in your Colab Notebook after running your detections, as well as upload and download files to and from Colab.

In [ ]:
# define helper functions

# function to display image
def imShow(path):
    import cv2
    import matplotlib.pyplot as plt
    %matplotlib inline

    image = cv2.imread(path)
    height, width = image.shape[:2]
    resized_image = cv2.resize(
        image, (3*width, 3*height), interpolation=cv2.INTER_CUBIC)

    fig = plt.gcf()
    fig.set_size_inches(18, 10)
    plt.axis("off")
    plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
    plt.show()


# function to download a file
def download(path):
    from google.colab import files
    files.download(path)


# function to upload a file
def upload():
    from google.colab import files
    uploaded = files.upload()
    for name, data in uploaded.items():
        with open(name, 'wb') as f:
            f.write(data)
            print('saved file', name)


## 4. Bring ANPD Dataset Into Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# unzip dataset into the anpd folder
!unzip /content/drive/MyDrive/Omdena_Abuja_ANPD/obj.zip -d /content/anpd/

In [ ]:
import glob

# make train.txt and test.txt file
file_train = open('/content/anpd/train.txt', 'w')
file_test = open('/content/anpd/test.txt', 'w')

img_list = glob.glob(f"/content/anpd/obj/*.jpg", recursive=False)
dataset_size = len(img_list)
train_pct = 0.9

train_idx = round(dataset_size * train_pct)
print(f"{len(img_list)} images found")
for i, img in enumerate(img_list, 1):
    file_train.write(img.replace("\\", "/") + "\n")

    if i > train_idx:
        file_test.write(img.replace("\\", "/") + "\n")

file_train.close()
file_test.close()

%cd  darknet/

## 5. Train ANPD

**TIP**: This training could take several hours depending on how many iterations you chose in the .cfg file. You will want to let this run as you sleep or go to work for the day, etc. However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Paste the following code into your console window and hit Enter
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```
Looks like this, it will click the screen every 10 minutes so that you don't get kicked off for being idle! HACKS!

*Double click to start training from best pre-trained weights*

<!-- remove the comment sign above and below
# starting training from last pre-trained weights [optional]

!./darknet detector train /content/anpd/anpd.data /content/anpd/anpd.cfg /content/anpd/out/anpd_best.weights -dont_show
change cell to code to run it-->

In [ ]:
# start training

!./darknet detector train /content/anpd/anpd.data /content/anpd/anpd.cfg /content/anpd/yolov4-tiny.conv.29 -dont_show -map

*Double click to resume training from last weights*

<!-- remove the comment sign above and below
# resuming training from last weights [optional]

!./darknet detector train /content/anpd/anpd.data /content/anpd/anpd.cfg /content/anpd/out/obj_last.weights - dont_show
chane cell to code to run it-->


In [ ]:
# show chart.png of how ANPD did with training

imShow('chart.png')

In [ ]:
# checking mAP

!./darknet detector map /content/anpd/anpd.data /content/anpd/anpd.cfg /content/anpd/out/anpd_best.weights

## 6. Test ANPD

In [ ]:
# need to set our custom cfg to test mode 

%cd /content/anpd/

!sed -i 's/batch=64/batch=1/' anpd.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' anpd.cfg

%cd /content/darknet/

In [ ]:
# upload test image
%cd ../

upload()

%cd darknet/

In [ ]:
# start testing

!./darknet detector test /content/anpd/anpd.data /content/anpd/anpd.cfg /content/anpd/out/anpd_best.weights /content/img4.jpg -thresh 0.3

imShow('predictions.jpg')

# Thanks!

[Abdulazeez Jimoh](https://www.linkedin.com/in/abdulazeez-oj)